### Spotipy - Get Latest Tracks

In this file the goal is to extract and process all the data about the latest songs played by a certain account.

In [1]:
# !pip install spotipy --user
# !pip install wordcloud --user
# !pip install pandas --user
# !pip install numpy --user
# !pip install matplotlib --user
# !pip install seaborn --user
# !pip install seaborn --user
# !pip install python-dotenv --user
# !pip install openpyxl --user
# !pip install pyarrow --user

#The lines below were necessary due to some bugs with the latest updates of spotipy
# !pip install git+https://github.com/plamere/spotipy.git --upgrade
# !pip install urllib3 --upgrade 
# !pip install requests --upgrade 
# !pip install spotipy --upgrade

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
from functions.spotipyTools import *
from dotenv import load_dotenv
import os
import openpyxl

In [3]:
load_dotenv()

True

In [5]:
#The instructions to replicate this step will be inluded in the repository readme file

e_client_id = os.environ["client_id"]
e_client_key = os.environ["client_key"]
e_playlist_id = os.environ["playlist_id"]
e_user_id = os.environ["user_id"]

sp_login = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=e_client_id,
                                               client_secret=e_client_key,
                                               redirect_uri="http://localhost:8888/callback", # Need to whitelist the URL
                                               scope="user-read-recently-played"))

First we use spotipy to grab all the the recently played tracks. We use the functions created in the _functions_ folder.

In [14]:
#The second argument is the number of tracks to fetch

all_tracks, dates_played = get_recent_tracks(sp_login, 300)

Extracts all the track's information

In [15]:
track_info_list = []

for item in all_tracks:
  track_info_list.append(get_track_info(item, sp_login))

df_tracks = pd.DataFrame(track_info_list, columns=['name', 'artist_name', 'album_name', 'album_date', 'album_popularity', 'track_duration', 'danceability', 'energy', 'instrumentalness', 'liveness', 'tempo','artist_url'])

df_tracks['date_played'] = dates_played

In [17]:
#Uncomment to append the new tracks to a previously existing list

#saved_tracks = pd.read_parquet('LatestPlayedTracks.parquet')
#df_tracks = pd.concat([df_tracks, saved_tracks]).drop_duplicates()

Exports to parquet to save for future analysis

In [18]:
df_tracks.to_parquet('LatestPlayedTracks.parquet', engine='pyarrow')